# Outline
Text goes here

In [148]:
# Imports
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from citipy import citipy
import random
import requests
import json
import io

# Get API key from api_keys.py
import api_keys
api_ow = api_keys.api_ow

In [149]:
# Sample API call for OpenWeather geolocation:
# http://api.openweathermap.org/geo/1.0/direct?q=CITYNAME,STATECODE,COUNTRYCODE&limit=LIMIT&appid=APIKEY

# Sample API call for OpenWeather weather data:
# units = "imperal"
#api_string = "https://api.openweathermap.org/data/2.5/weather?lat=" + LATITUDE + "&lon=" + LONGITUDE + "&appid=" + api_ow + "&units=" + units

In [150]:
# Functions

# Generates a random number between -180 and 180 degrees (0 is Greenwich, England).
def genLatitude():
    randomLat = random.uniform(-180, 180)
    return round(randomLat, 5)

# Generates a random number between -90 and 90 degrees (0 is equator)
def genLongitude():
    randomLong = random.uniform(-90, 90)
    return round(randomLong, 5)

# Send an OpenWeather API request with a city name, a country code, and a limit of the number of responses. In most cases the limit should be 1.
# Return the details of the city in a single-row dataframe.
def getCityCoordinates(city, country, limit):
    api_string = "http://api.openweathermap.org/geo/1.0/direct?q=" + city + "," + country + "&limit=" + str(limit) + "&appid=" + api_ow + "&lang=en"
    result = requests.get(api_string)
    print("Sending API request to geolocate city...")
    print("Got API response: " + result.content.decode("utf-8"))
    df_result = pd.read_json(result.content.decode("utf-8"))
    result_columns = ["name", "lat", "lon", "country", "state"]
    df_result = df_result.loc[:, result_columns]
    return df_result

# Send an OpenWeather API request with a set of latitude and longitude coordinates and the type of units (valid values are "standard", "metric", "imperial").
# Response will be in JSON. Extract temperature, humidity, cloudiness, wind speed.
# Return a single-row dataframe.
def getWeatherData(lat, long, units):
    api_string = "https://api.openweathermap.org/data/2.5/weather?lat=" + str(lat) + "&lon=" + str(long) + "&appid=" + api_ow + "&units=" + units
    result = requests.get(api_string)
    print("Sending API request for weather data...")
    print("Got API response: " + result.content.decode("utf-8"))
    json_result = result.content.decode("utf-8")
    json_result = json.loads(json_result)
    return json_result

In [151]:
# scraps

    #json_as_file = io.BytesIO(json_result)
    #dfs = []
    #with open(json_result, 'r') as f:
    #    for line in f:
    #        data = json.loads(line)
    #        df = pd.DataFrame.from_dict(data)
    #        dfs.append(df)
    #df_result = pd.concat(dfs, ignore_index=True)

    #df_result = pd.DataFrame.from_dict(data_json)
    #print(df_result)




    #df_result = pd.read_json(result.content.decode("utf-8"))
    #print(df_result)

In [152]:
# Generate 500 random sets of long/lat coordinates.
# Get the nearest city to each coordinates. If it's not in the list, add its weather data.
cities_500 = []
count = 1
while count <= 500:
    lat = genLatitude()
    long = genLongitude()
    nearest = citipy.nearest_city(lat, long)
    city = [nearest.city_name, nearest.country_code]

    # Only add the city if it's unique
    if city not in cities_500:
        cities_500.append(city)
        count += 1

In [153]:
# Dataframe to hold city data.
df_cities = pd.DataFrame(columns=["City Name", "Latitude", "Longitude", "Temperature (F)", "Humidity (%)", "Cloudiness (%)", "Wind speed (MPH)"])
df_cities

,City Name,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind speed (MPH)


In [154]:
# Use city name + country code to get exact coordinates, then use exact coordinates to get weather data.
# Then make dataframe row with weather data, then append dataframe row to df_cities.
df_test = getCityCoordinates(cities_500[0][0], cities_500[0][1], 1)
json_test = getWeatherData(df_test["lat"][0], df_test["lon"][0], "imperial")

row_to_add = {
    "City Name": json_test["name"],
    "Latitude": json_test["coord"]["lat"],
    "Longitude": json_test["coord"]["lon"],
    "Temperature (F)": json_test["main"]["temp"],
    "Humidity (%)": json_test["main"]["humidity"],
    "Cloudiness (%)": json_test["clouds"]["all"],
    "Wind Speed (MPH)": json_test["wind"]["speed"]
}
df_cities.loc[-1] = row_to_add
df_cities


Sending API request to geolocate city...
Got API response: [{"name":"Nicoya","lat":10.1488271,"lon":-85.4532657,"country":"CR"}]


KeyError: "['state'] not in index"

In [ ]:
json_test["wind"]["speed"]

7.07